In [ ]:
import requests
import time
import json
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ============================
# 1️⃣ Carregar dataset inicial
# ============================

# Substitua o caminho do arquivo pelo local onde está seu JSON inicial
with open('allcards.json', 'r', encoding='utf-8') as f:
    basic_data = json.load(f)

print(f"Número de cartas no dataset inicial: {len(basic_data)}")

In [ ]:
# ============================
# 2️⃣ Função para buscar dados completos na API
# ============================

def fetch_card_data(cardnumber, retries=3):
    url = f"https://digimoncard.io/api-public/search.php?card={cardnumber}"
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data:
                    return data[0]  # A API retorna uma lista com 1 elemento
                else:
                    print(f"⚠️ Nenhum dado retornado para {cardnumber}")
                    return None
            elif response.status_code == 429:
                # Too Many Requests: esperar mais tempo
                wait_time = 5 * (attempt + 1)
                print(f"🚦 Too Many Requests. Aguardando {wait_time} segundos...")
                time.sleep(wait_time)
            else:
                print(f"❌ Erro {response.status_code} para {cardnumber}")
                return None
        except Exception as e:
            print(f"❌ Erro na requisição para {cardnumber}: {e}")
            time.sleep(5)
    return None

In [ ]:
# ============================
# 3️⃣ Coletar dados com controle de taxa
# ============================

full_data = []
for card in tqdm(basic_data):
    cardnumber = card.get("cardnumber")
    if not cardnumber:
        continue
    data = fetch_card_data(cardnumber)
    if data:
        full_data.append(data)
    else:
        print(f"⚠️ Dados não encontrados para {cardnumber}")
    
    # Sleep entre chamadas para evitar bloqueio (ajuste conforme necessário)
    #time.sleep(1.0)

In [ ]:
# ============================
# 4️⃣ Salvar resultados
# ============================

# Salvar em CSV (transformando em DataFrame)
df = pd.json_normalize(full_data)
df.to_csv('../data/digimon_cards_raw.csv', index=False)

print("✅ Dados salvos em 'digimon_cards_raw.csv'")
